In [1]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage


In [2]:
from typing import TypedDict, Optional, List, Dict, Any

class EmailState(TypedDict):
    email: Dict[str, Any]
    is_spam: Optional[bool]
    draft_response: Optional[str]
    messages: List[Dict[str, Any]]


In [3]:
def check_spam(state: EmailState) -> EmailState:
    subject = state["email"]["subject"]
    state["is_spam"] = "oferta" in subject.lower()
    return state


In [4]:
builder = StateGraph(EmailState)
builder.add_node("check_spam", check_spam)
builder.set_entry_point("check_spam")
builder.set_finish_point("check_spam")  # o END si hay más nodos
graph = builder.compile()


In [5]:
initial_state = {
    "email": {"subject": "¡Gran oferta!", "body": "Compra ahora"},
    "messages": []
}
result = graph.invoke(initial_state)
print(result)


{'email': {'subject': '¡Gran oferta!', 'body': 'Compra ahora'}, 'is_spam': True, 'messages': []}


In [ ]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# Paso 1: Configurar el modelo
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="tu_clave_aquí")

# Paso 2: Nodo de razonamiento (Chain of Thought)
def razonamiento_node(state):
    mensaje_usuario = state["ticket"]
    prompt = [
        SystemMessage(content="Eres un asistente de soporte técnico que razona paso a paso."),
        HumanMessage(content=f"Analiza el siguiente ticket y explica paso a paso qué tipo de problema es:\n\n{mensaje_usuario}")
    ]
    razonamiento = llm(prompt).content
    return {"razonamiento": razonamiento, "ticket": mensaje_usuario}

# Paso 3: Nodo de clasificación
def clasificacion_node(state):
    razonamiento = state["razonamiento"]
    prompt = [
        SystemMessage(content="Eres un clasificador de tickets."),
        HumanMessage(content=f"Basado en el siguiente razonamiento, clasifica el ticket como 'problema técnico', 'facturación' u 'otro':\n\n{razonamiento}")
    ]
    categoria = llm(prompt).content
    return {"categoria": categoria, "razonamiento": razonamiento}

# Paso 4: Construir el grafo
builder = StateGraph()
builder.add_node("razonamiento", razonamiento_node)
builder.add_node("clasificacion", clasificacion_node)

builder.set_entry_point("razonamiento")
builder.add_edge("razonamiento", "clasificacion")
builder.add_edge("clasificacion", END)

graph = builder.compile()

# Paso 5: Ejecutar el flujo
entrada = {"ticket": "No puedo acceder a mi cuenta desde ayer. Me dice que mi contraseña es incorrecta."}
resultado = graph.invoke(entrada)

print("🧠 Razonamiento:\n", resultado["razonamiento"])
print("📂 Categoría asignada:\n", resultado["categoria"])


C:\Users\cmoncada\AppData\Local\Temp\ipykernel_26892\4130173885.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0)


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0, 'model...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

## Siguiente codigo

In [ ]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from datetime import datetime
import random

# Inicializar modelo
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="TU_API_KEY")

# Estado inicial
initial_state = {
    "ticket": "No puedo acceder a mi cuenta desde ayer. Me dice que mi contraseña es incorrecta.",
    "start_time": datetime.now(),
    "true_category": "problema técnico",  # Para evaluar accuracy
}

 Nodo de Razonamiento Inicial

In [ ]:
def razonamiento_node(state):
    prompt = [
        SystemMessage(content="Eres un asistente de soporte que razona paso a paso."),
        HumanMessage(content=f"Analiza este ticket y explica paso a paso:\n\n{state['ticket']}")
    ]
    razonamiento = llm(prompt).content
    return {**state, "razonamiento": razonamiento}


Nodo de Clasificación de Ticket

In [ ]:
def clasificacion_node(state):
    prompt = [
        SystemMessage(content="Clasifica el ticket como 'problema técnico', 'facturación' u 'otro'."),
        HumanMessage(content=f"Basado en este razonamiento:\n\n{state['razonamiento']}")
    ]
    categoria = llm(prompt).content.strip().lower()
    return {**state, "predicted_category": categoria}


Nodo de Escalamiento a Humanos

In [ ]:
def escalamiento_node(state):
    # Simulación: si el ticket contiene "no puedo" o "error", escalar
    escalar = any(palabra in state["ticket"].lower() for palabra in ["no puedo", "error", "bloqueado"])
    return {**state, "escalado": escalar}


Nodo de Registro de Métricas

In [ ]:
def metricas_node(state):
    end_time = datetime.now()
    duracion = (end_time - state["start_time"]).total_seconds()

    # Accuracy
    accuracy = 1 if state["predicted_category"] == state["true_category"] else 0

    # Tasa de escalamiento (1 si escalado, 0 si no)
    escalamiento = 1 if state.get("escalado") else 0

    return {
        **state,
        "tiempo_resolucion": duracion,
        "accuracy_clasificacion": accuracy,
        "tasa_escalamiento": escalamiento
    }


Nodo de Generación de Respuesta

In [ ]:
def respuesta_node(state):
    prompt = [
        SystemMessage(content="Genera una respuesta clara y útil para el cliente."),
        HumanMessage(content=f"Ticket: {state['ticket']}\nCategoría: {state['predicted_category']}")
    ]
    respuesta = llm(prompt).content
    return {**state, "respuesta": respuesta}


Construcción del Grafo LangGraph

In [ ]:
builder = StateGraph()
builder.add_node("razonamiento", razonamiento_node)
builder.add_node("clasificacion", clasificacion_node)
builder.add_node("escalamiento", escalamiento_node)
builder.add_node("metricas", metricas_node)
builder.add_node("respuesta", respuesta_node)

builder.set_entry_point("razonamiento")
builder.add_edge("razonamiento", "clasificacion")
builder.add_edge("clasificacion", "escalamiento")
builder.add_edge("escalamiento", "metricas")
builder.add_edge("metricas", "respuesta")
builder.add_edge("respuesta", END)

graph = builder.compile()
resultado = graph.invoke(initial_state)

# Mostrar resultados
for clave in ["razonamiento", "predicted_category", "escalado", "tiempo_resolucion", "accuracy_clasificacion", "tasa_escalamiento", "respuesta"]:
    print(f"{clave.upper()}:\n{resultado[clave]}\n")


## TERCERA OPCION

In [ ]:
pip install langchain openai pandas

In [ ]:
import pandas as pd
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from datetime import datetime

# Inicializar modelo
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="TU_API_KEY")

# Nodo 1: Preprocesamiento
def preprocesamiento_node(state):
    texto = f"Cliente de {state['customer_age']} años, con el producto {state['product_purchased']}. Asunto: {state['ticket_subject']}. Descripción: {state['ticket']}"
    return {**state, "texto_enriquecido": texto}

# Nodo 2: Razonamiento
def razonamiento_node(state):
    prompt = [
        SystemMessage(content="Eres un asistente de soporte que razona paso a paso."),
        HumanMessage(content=f"Analiza este ticket:\n\n{state['texto_enriquecido']}")
    ]
    razonamiento = llm(prompt).content
    return {**state, "razonamiento": razonamiento}

# Nodo 3: Clasificación
def clasificacion_node(state):
    prompt = [
        SystemMessage(content="Clasifica el ticket como 'problema técnico', 'facturación' u 'otro'."),
        HumanMessage(content=f"Basado en este razonamiento:\n\n{state['razonamiento']}")
    ]
    categoria = llm(prompt).content.strip().lower()
    return {**state, "predicted_category": categoria}

# Nodo 4: Escalamiento
def escalamiento_node(state):
    escalar = (
        state["ticket_priority"] == "High" and state["customer_satisfaction_rating"] < 3
    ) or any(p in state["ticket"].lower() for p in ["no puedo", "error", "bloqueado"])
    return {**state, "escalado": escalar}

# Nodo 5: Métricas
def metricas_node(state):
    duracion = (datetime.now() - state["start_time"]).total_seconds()
    accuracy = 1 if state["predicted_category"] == state["true_category"] else 0
    escalamiento = 1 if state.get("escalado") else 0
    return {
        **state,
        "tiempo_resolucion": duracion,
        "accuracy_clasificacion": accuracy,
        "tasa_escalamiento": escalamiento
    }

# Nodo 6: Generación de respuesta
def respuesta_node(state):
    prompt = [
        SystemMessage(content="Genera una respuesta clara y útil para el cliente."),
        HumanMessage(content=f"Ticket: {state['ticket']}\nCategoría: {state['predicted_category']}")
    ]
    respuesta = llm(prompt).content
    return {**state, "respuesta": respuesta}

# Construcción del grafo
builder = StateGraph()
builder.add_node("preprocesamiento", preprocesamiento_node)
builder.add_node("razonamiento", razonamiento_node)
builder.add_node("clasificacion", clasificacion_node)
builder.add_node("escalamiento", escalamiento_node)
builder.add_node("metricas", metricas_node)
builder.add_node("respuesta", respuesta_node)

builder.set_entry_point("preprocesamiento")
builder.add_edge("preprocesamiento", "razonamiento")
builder.add_edge("razonamiento", "clasificacion")
builder.add_edge("clasificacion", "escalamiento")
builder.add_edge("escalamiento", "metricas")
builder.add_edge("metricas", "respuesta")
builder.add_edge("respuesta", END)

graph = builder.compile()

# Leer el archivo CSV
df = pd.read_csv("customer_support_tickets.csv")

# Lista para guardar resultados
resultados = []

# Iterar sobre cada fila del dataset
for _, row in df.iterrows():
    estado_inicial = {
        "ticket": row["Ticket Description"],
        "customer_age": row["Customer Age"],
        "product_purchased": row["Product Purchased"],
        "ticket_subject": row["Ticket Subject"],
        "ticket_priority": row["Ticket Priority"],
        "customer_satisfaction_rating": row.get("Customer Satisfaction Rating", 3),
        "true_category": row["Ticket Type"].lower(),
        "start_time": datetime.now()
    }

    resultado = graph.invoke(estado_inicial)
    resultados.append(resultado)

# Guardar resultados en un nuevo archivo CSV
resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv("resultados_tickets.csv", index=False)

print("Procesamiento completado. Resultados guardados en 'resultados_tickets.csv'.")



In [1]:
import pandas as pd
from datetime import datetime

# Dataset ficticio
data = {
    "Ticket ID": [1, 2, 3],
    "Customer Name": ["Alice", "Bob", "Charlie"],
    "Customer Email": ["alice@example.com", "bob@example.com", "charlie@example.com"],
    "Customer Age": [28, 35, 40],
    "Customer Gender": ["Female", "Male", "Male"],
    "Product Purchased": ["Premium Account", "Basic Account", "Premium Account"],
    "Date of Purchase": ["2023-01-15", "2023-02-20", "2023-03-05"],
    "Ticket Type": ["Technical Issue", "Billing Inquiry", "Technical Issue"],
    "Ticket Subject": ["Cannot access account", "Incorrect billing amount", "Error in transaction"],
    "Ticket Description": [
        "I cannot access my account since yesterday. It says my password is incorrect.",
        "I was charged twice for my subscription.",
        "I tried to make a transaction but it failed with an error message."
    ],
    "Ticket Status": ["Open", "Closed", "Open"],
    "Resolution": ["", "Refund issued", ""],
    "Ticket Priority": ["High", "Medium", "High"],
    "Ticket Channel": ["Email", "Phone", "Web"],
    "First Response Time": [5, 10, 3],
    "Time to Resolution": [24, 48, 72],
    "Customer Satisfaction Rating": [2, 4, 3]
}

# Crear DataFrame
df = pd.DataFrame(data)

# Lista para guardar resultados
resultados = []

# Funciones de nodos
def preprocesamiento_node(row):
    texto = f"Cliente de {row['Customer Age']} años, con el producto {row['Product Purchased']}. Asunto: {row['Ticket Subject']}. Descripción: {row['Ticket Description']}"
    return {**row, "texto_enriquecido": texto}

def razonamiento_node(state):
    razonamiento = f"El cliente tiene {state['Customer Age']} años y usa {state['Product Purchased']}. El problema descrito es: {state['Ticket Description']}"
    return {**state, "razonamiento": razonamiento}

def clasificacion_node(state):
    if "password" in state["Ticket Description"].lower() or "transaction" in state["Ticket Description"].lower():
        categoria = "technical issue"
    elif "charged" in state["Ticket Description"].lower():
        categoria = "billing inquiry"
    else:
        categoria = "other"
    return {**state, "predicted_category": categoria}

def escalamiento_node(state):
    escalar = (
        state["Ticket Priority"] == "High" and state["Customer Satisfaction Rating"] < 3
    ) or any(p in state["Ticket Description"].lower() for p in ["no puedo", "error", "bloqueado"])
    return {**state, "escalado": escalar}

def metricas_node(state):
    duracion = state["Time to Resolution"]
    accuracy = 1 if state["predicted_category"] == state["Ticket Type"].lower() else 0
    escalamiento = 1 if state.get("escalado") else 0
    return {
        **state,
        "tiempo_resolucion": duracion,
        "accuracy_clasificacion": accuracy,
        "tasa_escalamiento": escalamiento
    }

def respuesta_node(state):
    respuesta = f"Estimado cliente, hemos clasificado su problema como '{state['predicted_category']}'. Estamos trabajando en ello."
    return {**state, "respuesta": respuesta}

# Iterar sobre cada fila del dataset
for _, row in df.iterrows():
    estado_inicial = row.to_dict()
    estado_inicial["start_time"] = datetime.now()

    # Ejecutar nodos
    estado = preprocesamiento_node(estado_inicial)
    estado = razonamiento_node(estado)
    estado = clasificacion_node(estado)
    estado = escalamiento_node(estado)
    estado = metricas_node(estado)
    estado = respuesta_node(estado)

    resultados.append(estado)

# Mostrar resultados
for resultado in resultados:
    for clave in ["texto_enriquecido", "razonamiento", "predicted_category", "escalado", "tiempo_resolucion", "accuracy_clasificacion", "tasa_escalamiento", "respuesta"]:
        print(f"{clave.upper()}:\n{resultado[clave]}\n")
    print("="*50)

# Guardar resultados en un nuevo CSV
#resultados_df = pd.DataFrame(resultados)
#resultados_df.to_csv("resultados_tickets.csv", index=False)



TEXTO_ENRIQUECIDO:
Cliente de 28 años, con el producto Premium Account. Asunto: Cannot access account. Descripción: I cannot access my account since yesterday. It says my password is incorrect.

RAZONAMIENTO:
El cliente tiene 28 años y usa Premium Account. El problema descrito es: I cannot access my account since yesterday. It says my password is incorrect.

PREDICTED_CATEGORY:
technical issue

ESCALADO:
True

TIEMPO_RESOLUCION:
24

ACCURACY_CLASIFICACION:
1

TASA_ESCALAMIENTO:
1

RESPUESTA:
Estimado cliente, hemos clasificado su problema como 'technical issue'. Estamos trabajando en ello.

TEXTO_ENRIQUECIDO:
Cliente de 35 años, con el producto Basic Account. Asunto: Incorrect billing amount. Descripción: I was charged twice for my subscription.

RAZONAMIENTO:
El cliente tiene 35 años y usa Basic Account. El problema descrito es: I was charged twice for my subscription.

PREDICTED_CATEGORY:
billing inquiry

ESCALADO:
False

TIEMPO_RESOLUCION:
48

ACCURACY_CLASIFICACION:
1

TASA_ESCALAM